In [28]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from pymongo import MongoClient

###########################
# Data Manipulation / Model
###########################
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "corey"
shelter = AnimalShelter(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class

#df = pd.DataFrame.from_records(shelter.read({}))
df = pd.read_csv('/usr/local/datasets/aac_shelter_outcomes.csv')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')
image_filename = 'GraziosoSalvareLogo.png' # Customer image 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard by Corey Denny'))),
    html.Hr(),
    # customer image location, can you put it in different location and resize?
    dcc.Link(html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'
                                  .format(encoded_image.decode()))), href='treset'),
    # Button location at top
    html.Div(dcc.RadioItems(
        options = [
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue', 'value': 'disasater'},
            {'label': 'Reset', 'value': 'reset'},
        ],
        value = "",
        labelStyle={'display': 'inline-block'}
        ),
    ),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),
                 
              html.Div(
           
    id='graph-id',
    className='col s12 m6',
    children = [
        dcc.Graph(id="graph",
                 
                 )
    ])   
        ]),
])
#############################################
# Interaction Between Components / Controller
#############################################

#---This code does not work but is progress towards being able to filter the data---#

#@app.callback([Output('datatable-id','data'),
#               Output('datatable-id','columns')],
#              [Input('radio-items', 'value')])
#def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
#        df = pd.read_csv('/usr/local/datasets/aac_shelter_outcomes.csv')

# if (value == 'reset'): 
#            df = pd.DataFrame.from_dict(list(shelter.read({})))
#        elif (value == 'water'):                     
#            df = pd.DataFrame.from_dict(shelter.read(
#                {"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix",
#                "Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female",
#                 "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#        return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    df = pd.read_csv('/usr/local/datasets/aac_shelter_outcomes.csv')
    return [
        dcc.Graph(            
            figure = px.bar(df, x="animal_type", y="age_upon_outcome_in_weeks")
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]




app
